In [ ]:
# %load createData.py

import re
import bs4
import requests
import logging
import csv
import xlwt, xlrd
from time import sleep

INDEX_URL = 'https://mojim.com'

def broswer_web(url):
    try:
        res = requests.get(url)
        page = bs4.BeautifulSoup(res.text, 'html.parser')
    except:
        print("ERROR REQUEST")

    return res, page

def is_chineseStr(string):
    for _char in string:
        if not '\u4e00' <= _char <= '\u9fa5':
            return False

    return True

def check_excel_format():
    workbook = xlrd.open_workbook('QA.xls')
    table = workbook.sheet_by_name("QA_data")
    question = []
    answer = []

    for count in range(1, table.nrows):
        print(table.cell_value(count, 0))
        print(table.cell_value(count, 1))
        print(table.cell_value(count, 2))
        print(table.cell_value(count, 3))
        print()

# specific web format
def read_excel(quesion_num=0, answer_num=1):
    workbook = xlrd.open_workbook('QA.xls')
    table = workbook.sheet_by_name("QA_data")
    question = []
    answer = []

    for count in range(1, table.nrows):
        question.append(table.cell_value(count, quesion_num))
        answer.append(table.cell_value(count, answer_num))
        # print(table.cell_value(count, 0))
        # print(table.cell_value(count, 1))
        # print(table.cell_value(count, 2))
        # print(table.cell_value(count, 3))
        # print()
    return question, answer

def read_csv():
    csvfile = "QA.csv"
    with open(csvfile,"r", encoding="utf-8", newline='') as f:
        csv_read = csv.reader(f)
        for line in csv_read:
            print(line)

class Music_Web(object):

    def __init__(self):
        self.url = INDEX_URL + '/twzhot-song.htm'
        self.res, self.page = broswer_web(self.url)
        self.music_list = []

    def merge_music(self):
        block = []
        title_set = {}

        # web format start from 2 to 2+100
        # for i in range(2, 2+100):
        for i in range(2, 2+20):
            block.append(self.page.select('#mx5_A > table > tr:nth-child(' + str(i) + ') > td'))

        # each block for 6 songs
        for music_6 in block:
            for music in music_6:
                url = music.select('a')[0]['href']
                title = re.sub('^\d*.', '', music.select('a')[0].string)
                singer = music.select('a')[1].string[2:-2]

                # check whether non-repetitive song
                if title in title_set:
                    continue
                s = Song(url, title, singer)
                if s.is_chinese_song():
                    self.music_list.append(s)

    def create_excel(self):
        workbook = xlwt.Workbook(encoding='utf-8')
        sheet1 = workbook.add_sheet("QA_data")
        sheet1.write(0, 0, 'feature(question)')
        sheet1.write(0, 1, 'title')
        sheet1.write(0, 2, 'singer')
        sheet1.write(0, 3, 'lyrcs(ans)')

        for count in range(0, len(self.music_list)):
            sheet1.write(count+1, 1, str(self.music_list[count].title))
            sheet1.write(count+1, 2, str(self.music_list[count].singer))
            sheet1.write(count+1, 3, str(self.music_list[count].lyrcs))
        workbook.save('QA_test.xls')

    def create_csv(self):
        csvfile = "QA_test.csv"
        with open(csvfile, 'w', encoding="utf-8", newline='') as f:
            csv_write = csv.writer(f)
            csv_write.writerow(['feature(question)', 'title', 'singer', 'lyrcs(ans)'])
            for song in self.music_list:
                csv_write.writerow(['', str(song.title), str(song.singer), str(song.lyrcs)])

class Song(object):

    def __init__(self, url, title, singer):
        self.url = INDEX_URL + url
        self.title = title
        self.singer = singer
        self.res, self.page = broswer_web(self.url)
        self.lyrcs = ""
        self.get_lyrcs()

    # get all lyrcs in keeping with chinese str
    def get_lyrcs(self):
        for string in re.split('<br/>|，| ', str(self.page.select('#fsZx3')[0])):

            if is_chineseStr(string):
                if not ('更多更詳盡歌詞' in string or '感謝' in string):
                    self.lyrcs += string

    # standard by enough chinese characters
    def is_chinese_song(self):
        return len(self.lyrcs) > 200

if __name__== '__main__':
    web = Music_Web()
    web.merge_music()
    web.create_csv()
    web.create_excel()
